In [1]:
rawData=sc.textFile('train_noheader.tsv')
records=rawData.map(lambda x:x.split('\t'))

In [2]:
records.first()

[u'"http://www.bloomberg.com/news/2010-12-23/ibm-predicts-holographic-calls-air-breathing-batteries-by-2015.html"',
 u'"4042"',
 u'"{""title"":""IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries"",""body"":""A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose California Photographer Tony Avelar Bloomberg By 2015 your mobile phone will project a 3 D image of anyone who calls and your laptop will be powered by kinetic energy At least that s what International Business Machines Corp sees in its crystal ball The predictions are part of an annual tradition for the Armonk New York based company which surveys its 3 000 researchers to find five ideas expected to take root in the next five years IBM the world 

In [3]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors

In [4]:
trimmed=records.map(lambda x:[xx.replace('\"','') for xx in x])

In [5]:
label=trimmed.map(lambda x:x[-1])

In [6]:
data=trimmed.map(lambda x:(x[-1],x[4:-1])).\
map(lambda (x,y):(x.replace('\"',''),[0.0 if yy=='\"?\"' else yy.replace('\"','') for yy in y])).\
map(lambda (x,y):(x.replace('\"',''),[0.0 if yy=='?' else yy.replace('\"','') for yy in y])).\
map(lambda (x,y):(int(x),[float(yy) for yy in y])).\
map(lambda (x,y):LabeledPoint(x,Vectors.dense(y)))

In [7]:
data.take(5)

[LabeledPoint(0.0, [0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575]),
 LabeledPoint(1.0, [0.574147,3.677966102,0.50802139,0.288770053,0.213903743,0.144385027,0.468648998,0.0,0.0,0.098707403,0.0,0.203489628,0.088652482,1.0,1.0,40.0,0.0,4973.0,187.0,9.0,0.181818182,0.125448029]),
 LabeledPoint(1.0, [0.996526,2.382882883,0.562015504,0.321705426,0.120155039,0.042635659,0.525448029,0.0,0.0,0.072447859,0.0,0.22640177,0.120535714,1.0,1.0,55.0,0.0,2240.0,258.0,11.0,0.166666667,0.057613169]),
 LabeledPoint(1.0, [0.801248,1.543103448,0.4,0.1,0.016666667,0.0,0.480724749,0.0,0.0,0.095860566,0.0,0.265655744,0.035343035,1.0,0.0,24.0,0.0,2737.0,120.0,5.0,0.041666667,0.100858369]),
 LabeledPoint(0.0, [0.719157,2.676470588,0.5,0.222222222,0.12345679,0.043209877,0.446143274,0.0,0.0,0.024908425,0.0,0.228887247,0.050473186,1.0,1.0,14.0,0.0,12032.0,162.0,10.0,0.09876543

In [8]:
data.cache()
numData=data.count()

In [9]:
numData

7395

In [10]:
nbdata=trimmed.map(lambda x:(x[-1],x[4:-1])).\
map(lambda (x,y):(x.replace('\"',''),[0.0 if yy=='\"?\"' else yy.replace('\"','') for yy in y])).\
map(lambda (x,y):(x.replace('\"',''),[0.0 if yy=='?' else yy.replace('\"','') for yy in y])).\
map(lambda (x,y):(int(x),[float(yy) for yy in y])).\
map(lambda (x,y):(x,[0.0 if yy<0 else yy for yy in y])).\
map(lambda (x,y):LabeledPoint(x,Vectors.dense(y)))

In [11]:
nbdata.first()

LabeledPoint(0.0, [0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575])

In [12]:
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.classification import SVMWithSGD
from pyspark.mllib.classification import NaiveBayes
from pyspark.mllib.tree import DecisionTree

In [13]:
numIteration=10
maxTreeDepth=5

In [14]:
numClass=label.distinct().count()

In [15]:
numClass

2

In [16]:
lrModel=LogisticRegressionWithSGD.train(data,numIteration)

/Users/qinkevin/spark/python/pyspark/mllib/classification.py:313: UserWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "Deprecated in 2.0.0. Use ml.classification.LogisticRegression or "


In [17]:
lrModel

(weights=[-0.110216274454,-0.493200344739,-0.0712665620384,-0.0214744216778,0.00276706475384,0.00246385887598,-1.33300460292,0.0525232672351,0.0,-0.0320576776,-0.00653638798541,-0.0613702511674,-0.14975863133,-0.13648187383,-0.121161700009,-15.6451616669,-0.0177690355464,745.987958686,-7.73567729685,-1.38587998188,-0.0355600416613,-0.0352085128613], intercept=0.0)

In [18]:
svmModel=SVMWithSGD.train(data,numIteration)

In [19]:
svmModel

(weights=[-0.122188386978,-0.527510758159,-0.0742371782434,-0.0206667449306,0.00546395033577,0.00409811283781,-1.54824523474,0.0607028905087,0.0,-0.037008323802,-0.007374037142,-0.067970375864,-0.172289581054,-0.148716595522,-0.129369384966,-18.0315472516,-0.0202704220321,1025.48043141,-5.05188911633,-1.54111193167,-0.038689478606,-0.0397619278886], intercept=0.0)

In [20]:
nbModel=NaiveBayes.train(nbdata)

In [21]:
nbModel

In [22]:
dtModel=DecisionTree.trainClassifier(data,numClass,{},impurity='entropy',maxDepth=maxTreeDepth)

In [23]:
dtModel

DecisionTreeModel classifier of depth 5 with 61 nodes

In [24]:
dataPoint=data.first()

In [25]:
prediction=lrModel.predict(dataPoint.features)

In [26]:
prediction

1

In [27]:
int(dataPoint.label)

0

In [28]:
predictions=lrModel.predict(data.map(lambda x:x.features))

In [29]:
predictions.take(10)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [30]:
data.map(lambda x:int(x.label)).take(10)

[0, 1, 1, 1, 0, 0, 1, 0, 1, 1]

In [31]:
lrTotalCorrect=data.map(lambda point:1 if(lrModel.predict(point.features)==point.label) else 0).sum()

In [32]:
svmTotalCorrect=data.map(lambda point:1 if(svmModel.predict(point.features)==point.label) else 0).sum()

In [33]:
nbTotalCorrect=data.map(lambda point:1 if(nbModel.predict(point.features)==point.label) else 0).sum()

In [34]:
predictLabel=dtModel.predict(data.map(lambda point:point.features)).collect()

In [35]:
trueLabel=data.map(lambda point:point.label).collect()

In [36]:
dtTotalCorrect=sum([1.0 if prediction==trueLabel[i] else 0.0 for i,prediction in enumerate(predictLabel)])

In [37]:
lrAccuracy=lrTotalCorrect/(data.count()*1.0)

In [38]:
lrAccuracy

0.5146720757268425

In [39]:
svmAccuracy=svmTotalCorrect/(data.count()*1.0)

In [40]:
svmAccuracy

0.5146720757268425

In [41]:
nbAccuracy=nbTotalCorrect/(data.count()*1.0)

In [42]:
nbAccuracy

0.5805273833671399

In [43]:
dtAccuracy=dtTotalCorrect/(data.count()*1.0)

In [44]:
dtAccuracy

0.6482758620689655

In [45]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [46]:
all_models_metrics=[]
for model in [lrModel,svmModel]:
    scoresAndLabels=data.map(lambda point:(model.predict(point.features),point.label)).collect()
    scoresAndLabels=[[float(i),j] for i,j in scoresAndLabels]
    
    rdd_scoresAndLabels=sc.parallelize(scoresAndLabels)
    metrics=BinaryClassificationMetrics(rdd_scoresAndLabels)
    all_models_metrics.append((model.__class__.__name__,metrics.areaUnderROC,metrics.areaUnderPR))        

In [47]:
all_models_metrics

[('LogisticRegressionModel', 0.5014181143280931, 0.7567586293858841),
 ('SVMModel', 0.5014181143280931, 0.7567586293858841)]

In [48]:
scoresAndLabels=nbdata.map(lambda point:(nbModel.predict(point.features),point.label)).collect()
scoresAndLabels=[[float(i),j] for i,j in scoresAndLabels]

rdd_scoresAndLabels=sc.parallelize(scoresAndLabels)
nb_metric=BinaryClassificationMetrics(rdd_scoresAndLabels)
(nbModel.__class__.__name__,nb_metric.areaUnderROC,nb_metric.areaUnderPR)

('NaiveBayesModel', 0.5835585110136261, 0.6808510815151734)

In [49]:
predictionLabels=dtModel.predict(data.map(lambda point:point.features)).collect()
trueLabels=data.map(lambda point:point.label).collect()
scoresAndLabels=[[prediction,trueLabel] for prediction,trueLabel in zip(predictionLabels,trueLabels)]
scoresAndLabels=[[float(i),j] for i,j in scoresAndLabels]
rdd_scoresAndLabels=sc.parallelize(scoresAndLabels)

dt_metric=BinaryClassificationMetrics(rdd_scoresAndLabels)
(dt_metric.__class__.__name__,dt_metric.areaUnderROC,dt_metric.areaUnderPR)

('BinaryClassificationMetrics', 0.6488371887050935, 0.7430805993331199)

In [50]:
from pyspark.mllib.stat import MultivariateStatisticalSummary
from pyspark.mllib.linalg.distributed import RowMatrix

vectors=data.map(lambda point:point.features)
matrix=MultivariateStatisticalSummary(vectors)

In [51]:
matrix.mean()

array([  4.12258053e-01,   2.76182319e+00,   4.68230473e-01,
         2.14079926e-01,   9.20623607e-02,   4.92621604e-02,
         2.25510345e+00,  -1.03750428e-01,   0.00000000e+00,
         5.64227450e-02,   2.12305612e-02,   2.33778177e-01,
         2.75709037e-01,   6.15551048e-01,   6.60311021e-01,
         3.00770791e+01,   3.97565923e-02,   5.71659824e+03,
         1.78754564e+02,   4.96064909e+00,   1.72864050e-01,
         1.01220792e-01])

In [52]:
matrix.min()

array([  2.17191000e-01,   1.42968750e+00,   2.62411348e-01,
         1.27659574e-01,   2.83687940e-02,   7.09219900e-03,
         5.16092603e-01,   0.00000000e+00,   0.00000000e+00,
         8.31889080e-02,   0.00000000e+00,   2.54468485e-01,
         1.98738170e-01,   1.00000000e+00,   0.00000000e+00,
         3.50000000e+01,   0.00000000e+00,   1.64100000e+03,
         1.41000000e+02,   5.00000000e+00,   3.54609930e-02,
         5.44871790e-02])

In [53]:
matrix.max()

array([  2.59910000e-01,   4.06250000e+00,   3.15789474e-01,
         3.15789474e-01,   2.63157895e-01,   0.00000000e+00,
         5.41310541e-01,   0.00000000e+00,   0.00000000e+00,
         8.20512820e-02,   0.00000000e+00,   2.68753233e-01,
         2.34375000e-01,   0.00000000e+00,   1.00000000e+00,
         5.20000000e+01,   0.00000000e+00,   3.02000000e+02,
         1.90000000e+01,   2.00000000e+00,   7.36842105e-01,
         0.00000000e+00])

In [54]:
matrix.count()

7395

In [55]:
matrix.variance()

array([  1.09727602e-01,   7.42907773e+01,   4.12575900e-02,
         2.15305244e-02,   9.21057177e-03,   5.27422016e-03,
         3.25347870e+01,   9.39571798e-02,   0.00000000e+00,
         1.71750875e-03,   2.07798245e-02,   2.75446690e-03,
         3.68329077e+00,   2.36647955e-01,   2.24300377e-01,
         4.15822321e+02,   3.81760057e-02,   7.87626486e+07,
         3.22037609e+04,   1.04515955e+01,   3.35890913e-02,
         6.27668400e-03])

In [56]:
from pyspark.mllib.feature import StandardScalerModel,StandardScaler

scaler=StandardScaler(withMean=True,withStd=True).fit(vectors)
labels=data.map(lambda point:point.label)
features=data.map(lambda point:point.features)
scaled_data=labels.zip(scaler.transform(features)).map(lambda (x,y):LabeledPoint(x,y))

In [57]:
scaled_data.map(lambda point:point.features).take(3)

[DenseVector([1.1376, -0.0819, 1.0251, -0.0559, -0.4689, -0.3543, -0.3175, 0.3385, 0.0, 0.8288, -0.1473, 0.2296, -0.1416, 0.7902, 0.7172, -0.298, -0.2035, -0.033, -0.0488, 0.9401, -0.1087, -0.2788]),
 DenseVector([0.4887, 0.1063, 0.1959, 0.509, 1.2695, 1.3097, -0.3132, 0.3385, 0.0, 1.0202, -0.1473, -0.5771, -0.0975, 0.7902, 0.7172, 0.4866, -0.2035, -0.0838, 0.0459, 1.2494, 0.0489, 0.3058]),
 DenseVector([1.7637, -0.044, 0.4617, 0.7334, 0.2927, -0.0912, -0.3032, 0.3385, 0.0, 0.3867, -0.1473, -0.1405, -0.0808, 0.7902, 0.7172, 1.2221, -0.2035, -0.3917, 0.4416, 1.868, -0.0338, -0.5504])]

In [58]:
features.take(3)

[DenseVector([0.7891, 2.0556, 0.6765, 0.2059, 0.0471, 0.0235, 0.4438, 0.0, 0.0, 0.0908, 0.0, 0.2458, 0.0039, 1.0, 1.0, 24.0, 0.0, 5424.0, 170.0, 8.0, 0.1529, 0.0791]),
 DenseVector([0.5741, 3.678, 0.508, 0.2888, 0.2139, 0.1444, 0.4686, 0.0, 0.0, 0.0987, 0.0, 0.2035, 0.0887, 1.0, 1.0, 40.0, 0.0, 4973.0, 187.0, 9.0, 0.1818, 0.1254]),
 DenseVector([0.9965, 2.3829, 0.562, 0.3217, 0.1202, 0.0426, 0.5254, 0.0, 0.0, 0.0724, 0.0, 0.2264, 0.1205, 1.0, 1.0, 55.0, 0.0, 2240.0, 258.0, 11.0, 0.1667, 0.0576])]

In [59]:
lrModel_scaled=LogisticRegressionWithSGD.train(scaled_data,numIteration)
lrPredictionLabel=lrModel_scaled.predict(scaled_data.map(lambda point:point.features))
trueLabel=scaled_data.map(lambda point:point.label)
lrTotalCorrect_scaled=sum([1.0 if prediction==label else 0.0 for prediction,label in zip(lrPredictionLabel.collect(),trueLabel.collect())])/scaled_data.count()

In [60]:
lrTotalCorrect_scaled

0.6209601081812035

In [61]:
scoresAndLabels=scaled_data.map(lambda point:(lrModel_scaled.predict(point.features),point.label)).collect()
scoresAndLabels=[(float(i),j) for (i,j) in scoresAndLabels]
scoresAndLabels_rdd=sc.parallelize(scoresAndLabels)
metrics=BinaryClassificationMetrics(scoresAndLabels_rdd)
(lrModel_scaled.__class__.__name__,metrics.areaUnderROC,metrics.areaUnderPR)

('LogisticRegressionModel', 0.6201898373011353, 0.7277006868096805)

In [79]:
category_dict={}
categories=records.map(lambda x:x[3]).distinct().zipWithIndex().collect()
for (x,y) in [(key.replace('\"',''),val) for (key,val) in categories]:
    category_dict[x]=y
num_categories=len(category_dict)

In [80]:
category_dict

{u'?': 2,
 u'arts_entertainment': 11,
 u'business': 3,
 u'computer_internet': 1,
 u'culture_politics': 6,
 u'gaming': 7,
 u'health': 4,
 u'law_crime': 5,
 u'recreation': 0,
 u'religion': 10,
 u'science_technology': 12,
 u'sports': 8,
 u'unknown': 9,
 u'weather': 13}

In [81]:
num_categories

14

In [89]:
def build_feature(x):
    import numpy as np
    numeric_feature=[0.0 if yy=='?' else yy for yy in [y.replace('\"','') for y in x[4:-1]]]
    category_feature=np.zeros(num_categories)
    category_index=category_dict[x[3].replace('\"','')]
    category_feature[category_index]=1
    label=x[-1].replace('\"','')
    feature=LabeledPoint(label,Vectors.dense(list(category_feature)+numeric_feature))
    return feature

In [90]:
category_data=trimmed.map(lambda x:build_feature(x))

In [91]:
category_data.take(5)

[LabeledPoint(0.0, [0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575]),
 LabeledPoint(1.0, [1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.574147,3.677966102,0.50802139,0.288770053,0.213903743,0.144385027,0.468648998,0.0,0.0,0.098707403,0.0,0.203489628,0.088652482,1.0,1.0,40.0,0.0,4973.0,187.0,9.0,0.181818182,0.125448029]),
 LabeledPoint(1.0, [0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.996526,2.382882883,0.562015504,0.321705426,0.120155039,0.042635659,0.525448029,0.0,0.0,0.072447859,0.0,0.22640177,0.120535714,1.0,1.0,55.0,0.0,2240.0,258.0,11.0,0.166666667,0.057613169]),
 LabeledPoint(1.0, [0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.801248,1.543103448,0.4,0.1,0.016666667,0.0,0.480724749,0.0,0.0,0.095860566,0.0,0.265655744,0.035343035,1.0,0.0,24.0,0.0,2737.0

In [97]:
category_labels=category_data.map(lambda point:point.label)
category_features=category_data.map(lambda point:point.features)
scaler2=StandardScaler(withMean=True,withStd=True).fit(category_features)
scaled_category_data=category_labels.zip(scaler2.transform(category_features)).map(lambda (x,y):LabeledPoint(x,y))

In [99]:
scaled_category_data.map(lambda point:point.features).take(5)

[DenseVector([-0.4464, -0.2042, -0.6808, 2.7207, -0.271, -0.0649, -0.2205, -0.1019, -0.2327, -0.0285, -0.0991, -0.3818, -0.2017, -0.0233, 1.1376, -0.0819, 1.0251, -0.0559, -0.4689, -0.3543, -0.3175, 0.3385, 0.0, 0.8288, -0.1473, 0.2296, -0.1416, 0.7902, 0.7172, -0.298, -0.2035, -0.033, -0.0488, 0.9401, -0.1087, -0.2788]),
 DenseVector([2.2397, -0.2042, -0.6808, -0.3675, -0.271, -0.0649, -0.2205, -0.1019, -0.2327, -0.0285, -0.0991, -0.3818, -0.2017, -0.0233, 0.4887, 0.1063, 0.1959, 0.509, 1.2695, 1.3097, -0.3132, 0.3385, 0.0, 1.0202, -0.1473, -0.5771, -0.0975, 0.7902, 0.7172, 0.4866, -0.2035, -0.0838, 0.0459, 1.2494, 0.0489, 0.3058]),
 DenseVector([-0.4464, -0.2042, -0.6808, -0.3675, 3.6896, -0.0649, -0.2205, -0.1019, -0.2327, -0.0285, -0.0991, -0.3818, -0.2017, -0.0233, 1.7637, -0.044, 0.4617, 0.7334, 0.2927, -0.0912, -0.3032, 0.3385, 0.0, 0.3867, -0.1473, -0.1405, -0.0808, 0.7902, 0.7172, 1.2221, -0.2035, -0.3917, 0.4416, 1.868, -0.0338, -0.5504]),
 DenseVector([-0.4464, -0.2042, -0.6

In [100]:
lrModel_category_scaled=LogisticRegressionWithSGD.train(scaled_category_data,num_categories)

In [101]:
lr_totalCorrect_category_scaled=scaled_category_data.\
    map(lambda point:1 if(lrModel_category_scaled.predict(point.features)==point.label) else 0).sum()

In [102]:
lr_accuracy_category_scaled=lr_totalCorrect_category_scaled/(1.0*data.count())

In [103]:
lr_accuracy_category_scaled

0.6659905341446923

In [108]:
scoresAndLabels=scaled_category_data.map(lambda point:(lrModel_category_scaled.predict(point.features),point.label)).collect()
scoresAndLabels=[(float(i),j) for (i,j) in scoresAndLabels]
scoresAndLabels_rdd=sc.parallelize(scoresAndLabels)
metrics=BinaryClassificationMetrics(scoresAndLabels_rdd)
(lrModel_category_scaled.__class__.__name__,metrics.areaUnderROC,metrics.areaUnderPR)

('LogisticRegressionModel', 0.6657028603250347, 0.7582877380197778)

In [110]:
category_nbdata=scaled_category_data
category_nbdata_nonegative=category_nbdata.map(lambda point:(point.label,point.features)).\
    map(lambda (x,y):(x,[0.0 if yy<0.0 else yy for yy in y])).\
    map(lambda (x,y):LabeledPoint(x,Vectors.dense(y)))

In [115]:
nb_category_model=NaiveBayes.train(category_nbdata_nonegative)
nb_category_total_correct=category_nbdata_nonegative.\
    map(lambda point:1 if (nb_category_model.predict(point.features)==point.label) else 0).sum()
nb_category_accuracy=nb_category_total_correct/(1.0*category_nbdata_nonegative.count())

In [116]:
nb_category_accuracy

0.652738336713996

In [122]:
scoresAndLabels=category_nbdata_nonegative.map(lambda point:(nb_category_model.predict(point.features),point.label)).collect()
scoresAndLabels=[(float(i),j) for (i,j) in scoresAndLabels]
scoresAndLabels_rdd=sc.parallelize(scoresAndLabels)
metrics=BinaryClassificationMetrics(scoresAndLabels_rdd)
(nb_category_model.__class__.__name__,metrics.areaUnderROC,metrics.areaUnderPR)

('NaiveBayesModel', 0.6514679174141278, 0.7520379357367744)

In [123]:
def train_with_params(input_data,model,reg_param,num_iter,step_size):
    model=model.train(input_data,iterations=num_iter,regParam=reg_param,step=step_size)
    return model

In [126]:
def create_metrics(tag,input_data,model):
    scoresAndLabels_rdd=input_data.map(lambda x:(model.predict(x.features)*1.0,x.label*1.0))
    metrics=BinaryClassificationMetrics(scoresAndLabels_rdd)
    return tag,metrics.areaUnderROC,metrics.areaUnderPR

In [128]:
for iteration_num in [1,5,10,50]:
    model=train_with_params(scaled_category_data,LogisticRegressionWithSGD,0.0,iteration_num,1.0)
    label,ROC,PR=create_metrics('%d iterations'%iteration_num,scaled_category_data,model)
    print '%s,ROC = %f,PR = %f'%(label,ROC,PR)

1 iterations,ROC = 0.649520,PR = 0.745886
5 iterations,ROC = 0.666161,PR = 0.758022
10 iterations,ROC = 0.665483,PR = 0.757964
50 iterations,ROC = 0.668143,PR = 0.760930


In [131]:
for step_size in [0.001,0.01,0.1,1.0,10.0]:
    model=train_with_params(scaled_category_data,LogisticRegressionWithSGD,0.0,10,step_size)
    label,ROC,PR=create_metrics('%2.3f 的步长'%step_size,scaled_category_data,model)
    print '%s,ROC = %f,PR = %f'%(label,ROC,PR)

0.001 的步长,ROC = 0.649659,PR = 0.745974
0.010 的步长,ROC = 0.649644,PR = 0.746017
0.100 的步长,ROC = 0.655211,PR = 0.750008
1.000 的步长,ROC = 0.665483,PR = 0.757964
10.000 的步长,ROC = 0.619228,PR = 0.725713


In [133]:
for reg_param in [0.001,0.01,0.1,1.0,10.0]:
    model=train_with_params(scaled_category_data,LogisticRegressionWithSGD,reg_param,10,0.01)
    label,ROC,PR=create_metrics('正则化系数%2.3f'%reg_param,scaled_category_data,model)
    print '%s,ROC=%f,PR=%f'%(label,ROC,PR)

正则化系数0.001,ROC=0.649644,PR=0.746017
正则化系数0.010,ROC=0.649644,PR=0.746017
正则化系数0.100,ROC=0.649644,PR=0.746017
正则化系数1.000,ROC=0.649644,PR=0.746017
正则化系数10.000,ROC=0.648979,PR=0.745492


In [139]:
def train_with_params_dt(input_data,impurity,maxTreeDepth):
    dt_model=DecisionTree.trainClassifier(input_data,numClass,{},impurity,maxDepth=maxTreeDepth)
    return dt_model

In [140]:
def create_metrics_dt(tag,input_data,model):
    predictLabel=model.predict(input_data.map(lambda point:point.features)).collect()
    trueLabel=input_data.map(lambda point:point.label).collect()
    scoresAndLabels=[(predictLabel[i],true_val) for i,true_val in enumerate(trueLabel)]
    scoresAndLabels_rdd=sc.parallelize(scoresAndLabels)
    scoresAndLabels_rdd=scoresAndLabels_rdd.map(lambda (x,y):(float(x),float(y)))
    dt_metrics = BinaryClassificationMetrics(scoresAndLabels_rdd)
    return tag,dt_metrics.areaUnderROC,dt_metrics.areaUnderPR

In [141]:
for depth in [1,2,3,4,5,10,20]:
    for im in ['entropy','gini']:
        model=train_with_params_dt(data,im,depth)
        tag,ROC,PR=create_metrics_dt('impurity:%s, %d maxTreeDepth:'%(im,depth),data,model)
        print '%s,AUC=%f,PR=%f'%(tag,ROC,PR)

impurity:entropy, 1 maxTreeDepth:,AUC=0.593268,PR=0.749004
impurity:gini, 1 maxTreeDepth:,AUC=0.593268,PR=0.749004
impurity:entropy, 2 maxTreeDepth:,AUC=0.616839,PR=0.725164
impurity:gini, 2 maxTreeDepth:,AUC=0.616839,PR=0.725164
impurity:entropy, 3 maxTreeDepth:,AUC=0.626070,PR=0.751813
impurity:gini, 3 maxTreeDepth:,AUC=0.626070,PR=0.751813
impurity:entropy, 4 maxTreeDepth:,AUC=0.636333,PR=0.749393
impurity:gini, 4 maxTreeDepth:,AUC=0.636333,PR=0.749393
impurity:entropy, 5 maxTreeDepth:,AUC=0.648837,PR=0.743081
impurity:gini, 5 maxTreeDepth:,AUC=0.648916,PR=0.742894
impurity:entropy, 10 maxTreeDepth:,AUC=0.762552,PR=0.829623
impurity:gini, 10 maxTreeDepth:,AUC=0.783709,PR=0.843469
impurity:entropy, 20 maxTreeDepth:,AUC=0.984537,PR=0.988522
impurity:gini, 20 maxTreeDepth:,AUC=0.988707,PR=0.991328


In [143]:
def train_with_params_nb(input_data,lambda_para):
    nb_model=NaiveBayes.train(input_data,lambda_para)
    return nb_model

In [146]:
def create_metrics_nb(tag,nbdata,model):
    scoresAndLabels=nbdata.map(lambda point:(float(model.predict(point.features)),point.label))
    nb_metrics=BinaryClassificationMetrics(scoresAndLabels)
    return tag,nb_metrics.areaUnderROC,nb_metrics.areaUnderPR

In [147]:
for lambda_para in [0.001,0.01,0.1,1.0,10.0]:
    model=train_with_params_nb(nbdata,lambda_para)
    tag,ROC,PR=create_metrics_dt('lambda=%f'%lambda_para,nbdata,model)
    print '%s,AUC=%f,PR=%f'%(tag,ROC,PR)

lambda=0.001000,AUC=0.583697,PR=0.680961
lambda=0.010000,AUC=0.583697,PR=0.680961
lambda=0.100000,AUC=0.583697,PR=0.680961
lambda=1.000000,AUC=0.583559,PR=0.680851
lambda=10.000000,AUC=0.583412,PR=0.680762


In [157]:
train_test_split=scaled_category_data.randomSplit([0.6,0.4],123)
train=train_test_split[0]
test=train_test_split[1]
for reg_para in [0.0,0.001,0.0025,0.005,0.01]:
    model = train_with_params(train,LogisticRegressionWithSGD,reg_para,0.0,1.0)
    label,ROC,PR=create_metrics('%f regularization parameter '%reg_para,test,model)
    print '%s,AUC = %f,PR = %f'%(label,ROC,PR)

0.000000 regularization parameter ,AUC = 0.500000,PR = 0.757776
0.001000 regularization parameter ,AUC = 0.500000,PR = 0.757776
0.002500 regularization parameter ,AUC = 0.500000,PR = 0.757776
0.005000 regularization parameter ,AUC = 0.500000,PR = 0.757776
0.010000 regularization parameter ,AUC = 0.500000,PR = 0.757776
